# Cloning torchserve to add in the docker

In [1]:
# The dockerfile places the serve folder into the docker image
!git clone https://github.com/pytorch/serve.git

Cloning into 'serve'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 9454 (delta 12), reused 4 (delta 4), pack-reused 9427
Receiving objects: 100% (9454/9454), 38.27 MiB | 12.40 MiB/s, done.
Resolving deltas: 100% (5166/5166), done.


# Creating a docker image
### This creates a docker image from the dockerfile present in the current directory and pushes it to the ecr

In [8]:
import boto3, time, json
sess    = boto3.Session()
region  = sess.region_name
account = boto3.client('sts').get_caller_identity().get('Account')

In [9]:
import sagemaker

sagemaker_session = sagemaker.Session(boto_session=sess)

In [10]:
bucket_name = sagemaker_session.default_bucket()
print(bucket_name)
#bucket_name = "sagemaker-ap-southeast-1-614857483369"

sagemaker-us-west-1-096166425824


In [11]:
registry_name = 'torchserve2'
!aws ecr create-repository --repository-name {registry_name}


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'torchserve2' already exists in the registry with id '096166425824'


In [13]:
image_label = 'v1'
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{registry_name}:{image_label}'

!sudo docker build -t {registry_name}:{image_label} .
!$(aws ecr get-login --no-include-email --region {region})
!docker tag {registry_name}:{image_label} {image}
!docker push {image}

Sending build context to Docker daemon  69.01MB
Step 1/25 : FROM ubuntu:18.04
 ---> 2eb2d388e1a2
Step 2/25 : ENV PYTHONUNBUFFERED TRUE
 ---> Using cache
 ---> 1e6af8e44e06
Step 3/25 : RUN apt-get update &&     DEBIAN_FRONTEND=noninteractive apt-get install --no-install-recommends -y     fakeroot     ca-certificates     dpkg-dev     g++     python3-dev     openjdk-11-jdk     curl     vim     && rm -rf /var/lib/apt/lists/*     && cd /tmp     && curl -O https://bootstrap.pypa.io/get-pip.py     && python3 get-pip.py
 ---> Using cache
 ---> b40c3cd92164
Step 4/25 : RUN update-alternatives --install /usr/bin/python python /usr/bin/python3 1
 ---> Using cache
 ---> ee67aa137579
Step 5/25 : RUN update-alternatives --install /usr/local/bin/pip pip /usr/local/bin/pip3 1
 ---> Using cache
 ---> 32951d11ad09
Step 6/25 : RUN pip install aiohttp
 ---> Using cache
 ---> ed80cae964c1
Step 7/25 : RUN pip install allennlp==0.8.4
 ---> Using cache
 ---> 7a345e57d0b4
Step 8/25 : RUN pip install zmq
 ---> 

f2f01f9d: Pushing  1.643GB/3.282GBPushing  14.47MB/3.282GBPushing  213.9MB/3.282GBPushing  251.9MB/882.1MBPushing  1.269GB/3.282GB

f2f01f9d: Pushed   3.304GB/3.282GBPushing  2.009GB/3.282GBPushing  2.452GB/3.282GBv1: digest: sha256:428a17005950ec0d1b17369f512620061206a9ea19cff8f0992169f26eeb944a size: 5144
